## Regression of home prices

#### Libraries

In [167]:
import pandas as pd
import numpy as np
import scipy as sp
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.cross_validation import train_test_split

#### Load data

In [168]:
DATADIR = './Data/'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'

In [169]:
train_raw = pd.read_csv(DATADIR+TRAIN_FILE)
test_raw = pd.read_csv(DATADIR+TEST_FILE)

In [170]:
IDS = test_raw['Id']
del test_raw['Id']
del train_raw['Id']

In [171]:
Y = np.log(train_raw['SalePrice'].as_matrix())
del train_raw['SalePrice']

#### Create one-hot representation 

In [172]:
dtypes = train_raw.dtypes
categoricals = []
for i, f in enumerate(train_raw.columns):
    if dtypes[f] == np.dtype('O'):
        le = LabelEncoder()
        concat = (train_raw[f],test_raw[f])
        le.fit(np.hstack(concat))
        train_raw[f] = le.transform(train_raw[f])
        test_raw[f] = le.transform(test_raw[f])
        categoricals.append(i)

train_raw = train_raw.fillna(0)
test_raw = test_raw.fillna(0)
ohe = OneHotEncoder(categorical_features=categoricals)
ohe.fit(np.vstack((train_raw,test_raw)))
X = ohe.transform(train_raw)
X_test = ohe.transform(test_raw)

#### Scale data

In [173]:
ss = StandardScaler()
X = ss.fit_transform(X.A)
X_test = ss.transform(X_test.A)

#### Train-test split

In [174]:
# X_train, X_val, y_train, y_val = train_test_split( X, Y, test_size=0.25)
X_train, X_val, y_train, y_val = X[:1000], X[1000:], Y[:1000], Y[1000:]

In [175]:
dtrain = xgb.DMatrix( X_train, label=y_train )
dval = xgb.DMatrix( X_val, label=y_val )
dtest = xgb.DMatrix( X_test )

#### Boosted trees model 

In [176]:
d = 5
e = 0.01
t = 2000
param = {'max_depth':d, 
         'eta':e, 
         'subsample':0.5, 
         'colsample_bytree':0.5,
         'colsample_bylevel':0.5,
         'silent':1, 
         'lambda':1.0,
         'objective':'reg:linear' }
param['eval_metric'] = 'rmse'
param['nthread'] = 2
evallist  = [(dval,'eval'), (dtrain,'train')]
xgb_model = xgb.train(param.items(), dtrain, t+1, evallist, verbose_eval=t//10)

[0]	eval-rmse:11.403349	train-rmse:11.421618
[200]	eval-rmse:1.546024	train-rmse:1.562984
[400]	eval-rmse:0.251405	train-rmse:0.247601
[600]	eval-rmse:0.127124	train-rmse:0.091496
[800]	eval-rmse:0.118829	train-rmse:0.071480
[1000]	eval-rmse:0.116334	train-rmse:0.061970
[1200]	eval-rmse:0.115282	train-rmse:0.054581
[1400]	eval-rmse:0.114454	train-rmse:0.048318
[1600]	eval-rmse:0.113940	train-rmse:0.042790
[1800]	eval-rmse:0.113670	train-rmse:0.038013
[2000]	eval-rmse:0.113361	train-rmse:0.034047


In [177]:
predictions = xgb_model.predict(dval)
score = np.sqrt(np.mean((predictions-y_val)**2))
print 'Predicted RMSE (of log-values): {}'.format(round(score,4))

Predicted RMSE (of log-values): 0.1134


#### Re-predict with full dataset

In [178]:
dtrain = xgb.DMatrix( X, label=Y )
xgb_model = xgb.train(param.items(), dtrain, t+1)
predictions = xgb_model.predict(dtest)

In [179]:
submission = pd.DataFrame(np.vstack((IDS.astype(str), 
                                     np.exp(predictions))).T,
                          columns=['Id','SalePrice'])
submission.to_csv('submission.csv', index=False)